<a href="https://colab.research.google.com/github/anipnwr7777/music_generation_using_deep_learning/blob/main/lstm_music_genaration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding


In [ ]:
data_directory = "/content/my_proj"
data_file = "Data_Tunes.txt"
charIndex_json = "char_to_index.json"
model_weights_directory = "/content/my_proj/Model_Weights"
BATCH_SIZE = 16
SEQ_LENGTH = 64

In [ ]:
def read_batches(all_chars, unique_chars):
    length = all_chars.shape[0]
    batch_chars = int(length / BATCH_SIZE) 
    print(length)

    
    for start in range(0, batch_chars - SEQ_LENGTH, 64):
        X = np.zeros((BATCH_SIZE, SEQ_LENGTH))    
        Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, unique_chars))  
        for batch_index in range(0, 16):  
            for i in range(0, 64): 
                X[batch_index, i] = all_chars[batch_index * batch_chars + start + i]
                Y[batch_index, i, all_chars[batch_index * batch_chars + start + i + 1]] = 1
        yield X, Y

In [ ]:
def built_model(batch_size, seq_length, unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (batch_size, seq_length))) 
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))

    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))

    
    model.add(TimeDistributed(Dense(unique_chars)))

    model.add(Activation("softmax"))
    
    return model

In [ ]:

def training_model(data, epochs = 5):
    
    char_to_index = {ch: i for (i, ch) in enumerate(sorted(list(set(data))))}
    print("Number of unique characters in our whole tunes database = {}".format(len(char_to_index))) 
    
    with open(os.path.join(data_directory, charIndex_json), mode = "w") as f:
        json.dump(char_to_index, f)
        
    index_to_char = {i: ch for (ch, i) in char_to_index.items()}
    unique_chars = len(char_to_index)
    
    model = built_model(BATCH_SIZE, SEQ_LENGTH, unique_chars)
    model.summary()
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    all_characters = np.asarray([char_to_index[c] for c in data], dtype = np.int32)
    print("Total number of characters = "+str(all_characters.shape[0])) 
    
    epoch_number, loss, accuracy = [], [], []
    
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1, epochs))
        final_epoch_loss, final_epoch_accuracy = 0, 0
        epoch_number.append(epoch+1)
        
        for i, (x, y) in enumerate(read_batches(all_characters, unique_chars)):
            final_epoch_loss, final_epoch_accuracy = model.train_on_batch(x, y) 
            print("Batch: {}, Loss: {}, Accuracy: {}".format(i+1, final_epoch_loss, final_epoch_accuracy))
           
        loss.append(final_epoch_loss)
        accuracy.append(final_epoch_accuracy)
        
       
        if (epoch + 1) % 10 == 0:
            if not os.path.exists(model_weights_directory):
                os.makedirs(model_weights_directory)
            model.save_weights(os.path.join(model_weights_directory, "Weights_{}.h5".format(epoch+1)))
            print('Saved Weights at epoch {} to file Weights_{}.h5'.format(epoch+1, epoch+1))
    
   
    log_frame = pd.DataFrame(columns = ["Epoch", "Loss", "Accuracy"])
    log_frame["Epoch"] = epoch_number
    log_frame["Loss"] = loss
    log_frame["Accuracy"] = accuracy
    log_frame.to_csv("/content/my_proj/log.csv", index = False)


In [ ]:
file = open(os.path.join(data_directory, data_file), mode = 'r')
data = file.read()
file.close()
if __name__ == "__main__":
    training_model(data)

Number of unique characters in our whole tunes database = 88
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (16, 64, 512)             45056     
_________________________________________________________________
lstm_3 (LSTM)                (16, 64, 256)             787456    
_________________________________________________________________
dropout_3 (Dropout)          (16, 64, 256)             0         
_________________________________________________________________
lstm_4 (LSTM)                (16, 64, 256)             525312    
_________________________________________________________________
dropout_4 (Dropout)          (16, 64, 256)             0         
_________________________________________________________________
lstm_5 (LSTM)                (16, 64, 256)             525312    
___________________________________________________________

In [ ]:
log = pd.read_csv(os.path.join(data_directory, "log.csv"))
log

,Epoch,Loss,Accuracy
0,1,2.223322,0.385742
1,2,1.767112,0.473633
2,3,1.565153,0.517578
3,4,1.470443,0.541016
4,5,1.387324,0.565430
...,...,...,...
195,196,0.212401,0.928711
196,197,0.192337,0.943359
197,198,0.199914,0.935547
198,199,0.199582,0.940430


In [ ]:
length


NameError: ignored

In [ ]:
def make_model(unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (1, 1))) 
  
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, stateful = True)) 
   
    model.add(Dropout(0.2))
    
    model.add((Dense(unique_chars)))
    model.add(Activation("softmax"))
    
    return model

In [ ]:
def generate_sequence(epoch_num, initial_index, seq_length):
    with open(os.path.join(data_directory, charIndex_json)) as f:
        char_to_index = json.load(f)
    index_to_char = {i:ch for ch, i in char_to_index.items()}
    unique_chars = len(index_to_char)
    
    model = make_model(unique_chars)
    model.load_weights(os.path.join(model_weights_directory , "Weights_{}.h5".format(epoch_num)))
     
    sequence_index = [initial_index]
    
    for _ in range(seq_length):
        batch = np.zeros((1, 1))
        batch[0, 0] = sequence_index[-1]
        
        predicted_probs = model.predict_on_batch(batch).ravel()
        sample = np.random.choice(range(unique_chars), size = 1, p = predicted_probs)
        
        sequence_index.append(sample[0])
    
    seq = ''.join(index_to_char[c] for c in sequence_index)
    
    cnt = 0
    for i in seq:
        cnt += 1
        if i == "\n":
            break
    seq1 = seq[cnt:]
   
    
    cnt = 0
    for i in seq1:
        cnt += 1
        if i == "\n" and seq1[cnt] == "\n":
            break
    seq2 = seq1[:cnt]
    
    
    return seq2

In [ ]:
ep = int(input("1. Which epoch number weight you want to load into the model(10, 20, 30, ..., 90). Small number will generate more errors in music: "))
ar = int(input("\n2. Enter any number between 0 to 86 which will be given as initial charcter to model for generating sequence: "))
ln = int(input("\n3. Enter the length of music sequence you want to generate. Typical number is between 300-600. Too small number will generate hardly generate any sequence: "))

music = generate_sequence(ep, ar, ln)

print("\nMUSIC SEQUENCE GENERATED: \n")

print(music)

1. Which epoch number weight you want to load into the model(10, 20, 30, ..., 90). Small number will generate more errors in music: 200

2. Enter any number between 0 to 86 which will be given as initial charcter to model for generating sequence: 69

3. Enter the length of music sequence you want to generate. Typical number is between 300-600. Too small number will generate hardly generate any sequence: 1500

MUSIC SEQUENCE GENERATED: 

"A"cBA E2E|"D"F2A def|"Em"g3 "Bm"f2e|"Em"dcd "A7"ecA|\
"D"d3 "A7"d2:|
P:B
c/2d/2|"D"f2a a3|"A7"fgf ecA|"D"faf "G"g2f|"D/f+"a3 "G"b2g|"D"f2f fed|"Em"g2e "A7"f2g|
"D"a2f "A7"g2e|"D"a2f "G"b2g|"D"afd "Bm"d2f|"Em"e2B "Bm"B2d|"A7"cBc "D"d2:|
P:B
c/2d/2|"C"e2e ece|"G"d2g "Am"e2c|"G"dcB "D7"AGF|"G"G3 G2:|
K:D
P:B
f/2g/2|"D"a2f d2e|"D"fgf "A/c+"e2e|"Bm"d2d "B7/a"B2c|"Em"ded "A7"cBA|"D"d3 d2:|

